<a href="https://colab.research.google.com/github/geomyll33/Beautez/blob/master/keras_multivariate_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import modules and packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

%matplotlib inline

In [2]:
# Importing Training Set
dataset_train = pd.read_csv('/content/527163_ΟΗ_Μ.csv')
dataset_train=dataset_train.drop(['provision_id','meter_id'],axis=1)



In [3]:
dataset_train

,timestamp,indication,meter_var_normal,meter_var_stop,provision_var_change,provision_var_normal
0,15-03-01,18406,1,0,0,1
1,15-04-01,18831,1,0,0,1
2,15-05-01,19161,1,0,0,1
3,15-06-01,19541,1,0,0,1
4,15-07-01,19906,1,0,0,1
...,...,...,...,...,...,...
230,15-05-20,1657,1,0,0,1
231,15-06-20,1836,1,0,0,1
232,15-07-20,2029,1,0,0,1
233,15-08-20,2197,1,0,0,1


In [4]:
# Select features (columns) to be involved intro training and predictions
cols = list(dataset_train)[1:5]

# Extract dates (will be used in visualization)
datelist_train = list(dataset_train['timestamp'])
datelist_train = [dt.datetime.strptime(date, '%y-%m-%d').date() for date in datelist_train]

print('Training set shape == {}'.format(dataset_train.shape))
print('All timestamps == {}'.format(len(datelist_train)))
print('Featured selected: {}'.format(cols))

Training set shape == (235, 6)
All timestamps == 235
Featured selected: ['indication', 'meter_var_normal', 'meter_var_stop', 'provision_var_change']


In [5]:
dataset_train = dataset_train[cols].astype(str)
for i in cols:
    for j in range(0, len(dataset_train)):
        dataset_train[i][j] = dataset_train[i][j].replace(',', '')

dataset_train = dataset_train.astype(float)

# Using multiple features (predictors)
training_set = dataset_train.to_numpy()

print('Shape of training set == {}.'.format(training_set.shape))
training_set

Shape of training set == (235, 4).


array([[1.8406e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [1.8831e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [1.9161e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [1.9541e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [1.9906e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [1.9906e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.0370e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.0710e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.1010e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.1461e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.1766e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.2231e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.2575e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.2957e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.3397e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.3656e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.3942e+04, 1.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.4307e+04, 1.0000e+00,

In [6]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
training_set_scaled = sc.fit_transform(training_set)

sc_predict = StandardScaler()
sc_predict.fit_transform(training_set[:, 0:1])

array([[-1.38916929],
       [-1.37342577],
       [-1.36120139],
       [-1.34712484],
       [-1.33360393],
       [-1.33360393],
       [-1.31641571],
       [-1.3038209 ],
       [-1.29270783],
       [-1.27600118],
       [-1.26470289],
       [-1.24747763],
       [-1.23473464],
       [-1.22058399],
       [-1.20428482],
       [-1.19469053],
       [-1.18409607],
       [-1.17057517],
       [-1.156832  ],
       [-1.14597824],
       [-1.13190168],
       [-1.11808443],
       [-1.10800857],
       [-1.08893113],
       [-1.0752991 ],
       [-1.05674027],
       [-1.04284893],
       [-1.03166177],
       [-1.01943739],
       [-1.00691666],
       [-0.99447002],
       [-0.98168999],
       [-0.97372562],
       [-0.95753758],
       [-0.94975843],
       [-0.92912516],
       [-0.91653035],
       [-0.90549136],
       [-0.8874141 ],
       [-0.87518972],
       [-0.85651976],
       [-0.84629573],
       [-0.83744232],
       [-0.82336576],
       [-0.81221564],
       [-0

In [8]:

# Creating a data structure with 90 timestamps and 1 output
X_train = []
y_train = []

n_future = 12   # Number of days we want top predict into the future
n_past = 48     # Number of past days we want to use to predict the future

for i in range(n_past, len(training_set_scaled) - n_future +1):
    X_train.append(training_set_scaled[i - n_past:i, 0:dataset_train.shape[1] - 1])
    y_train.append(training_set_scaled[i + n_future - 1:i + n_future, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

print('X_train shape == {}.'.format(X_train.shape))
print('y_train shape == {}.'.format(y_train.shape))

X_train shape == (176, 48, 3).
y_train shape == (176, 1).


In [9]:

# Import Libraries and packages from Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import Adam

In [10]:
# Initializing the Neural Network based on LSTM
model = Sequential()

# Adding 1st LSTM layer
model.add(LSTM(units=64, return_sequences=True, input_shape=(n_past, dataset_train.shape[1]-1)))

# Adding 2nd LSTM layer
model.add(LSTM(units=10, return_sequences=False))

# Adding Dropout
model.add(Dropout(0.25))

# Output layer
model.add(Dense(units=1, activation='linear'))

# Compiling the Neural Network
model.compile(optimizer = Adam(learning_rate=0.01), loss='mean_squared_error')

In [11]:
%%time
es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
mcp = ModelCheckpoint(filepath='weights.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)

tb = TensorBoard('logs')

history = model.fit(X_train, y_train, shuffle=True, epochs=30, callbacks=[es, rlr, mcp, tb], validation_split=0.2, verbose=1, batch_size=256)

Epoch 1/30
1/1 [==============================] - 5s 5s/step - loss: 0.4838 - val_loss: 3.6151

Epoch 00001: val_loss improved from inf to 3.61507, saving model to weights.h5
Epoch 2/30
1/1 [==============================] - 0s 394ms/step - loss: 0.0928 - val_loss: 5.8183

Epoch 00002: val_loss did not improve from 3.61507
Epoch 3/30
1/1 [==============================] - 0s 169ms/step - loss: 0.2022 - val_loss: 4.3990

Epoch 00003: val_loss did not improve from 3.61507
Epoch 4/30
1/1 [==============================] - 0s 172ms/step - loss: 0.0358 - val_loss: 3.6147

Epoch 00004: val_loss improved from 3.61507 to 3.61471, saving model to weights.h5
Epoch 5/30
1/1 [==============================] - 0s 157ms/step - loss: 0.0409 - val_loss: 3.3931

Epoch 00005: val_loss improved from 3.61471 to 3.39309, saving model to weights.h5
Epoch 6/30
1/1 [==============================] - 0s 156ms/step - loss: 0.0823 - val_loss: 3.4115

Epoch 00006: val_loss did not improve from 3.39309
Epoch 7/30


In [12]:

# Generate list of sequence of days for predictions
datelist_future = pd.date_range(datelist_train[-1], periods=n_future, freq='1d').tolist()

'''
Remeber, we have datelist_train from begining.
'''

# Convert Pandas Timestamp to Datetime object (for transformation) --> FUTURE
datelist_future_ = []
for this_timestamp in datelist_future:
    datelist_future_.append(this_timestamp.date())

In [13]:

# Perform predictions
predictions_future = model.predict(X_train[-n_future:])

predictions_train = model.predict(X_train[n_past:])

In [19]:
# Inverse the predictions to original measurements

# ---> Special function: convert <datetime.date> to <Timestamp>
def datetime_to_timestamp(x):
    '''
        x : a given datetime value (datetime.date)
    '''
    return datetime.strptime(x.strftime('%y-%m-%d'), '%y-%m-%d')


y_pred_future = sc_predict.inverse_transform(predictions_future)
y_pred_train = sc_predict.inverse_transform(predictions_train)

PREDICTIONS_FUTURE = pd.DataFrame(y_pred_future, columns=['Open']).set_index(pd.Series(datelist_future))
PREDICTION_TRAIN = pd.DataFrame(y_pred_train, columns=['Open']).set_index(pd.Series(datelist_train[2 * n_past + n_future -1:]))

# Convert <datetime.date> to <Timestamp> for PREDCITION_TRAIN
PREDICTION_TRAIN.index = PREDICTION_TRAIN.index.to_series().apply(datetime_to_timestamp)

PREDICTION_TRAIN

,Open
2015-02-10,57450.046875
2015-03-10,57780.816406
2015-04-10,58112.574219
2015-05-10,58438.457031
2015-06-10,58753.265625
...,...
2015-05-20,89001.546875
2015-06-20,89171.382812
2015-07-20,89220.031250
2015-08-20,87133.585938
